# ThinCurr Python Example: XML Creation 

In this notebook we demonstrate how to utilize the filaments.py script from the ThinCurr package to help create oft xml files for user defined coils. 

## Load ThinCurr library

To load the ThinCurr filaments python script, you will need to define where ThinCurr is located. This can be done through the PYTHONPATH enviornment variable or within the script using sys.path.append() as shown below, where we look for the environment variable OFT_ROOTPATH to procide the path to where the OpenFUSIONToolkit is installed. If this enviornment variable is not defined, you can do so within the script as well by setting:
 
 
 ```os.environ["OFT_ROOTPATH"] = \path\to\OpenFUSIONToolkit\install_release```


In [ ]:
import os 
import sys
import numpy as np
thincurr_python_path = os.getenv('OFT_ROOTPATH')
if thincurr_python_path is not None:
    sys.path.append(os.path.join(thincurr_python_path,'python'))
from OpenFUSIONToolkit._core import OFT_env
from OpenFUSIONToolkit.ThinCurr.filaments import ThinCurr_Icoil, ThinCurr_Vcoil, ThinCurr_XML

## Creating a Filament array
In the following code block we will look at how to create a simple filament array using the ThinCurr_Icoil class and how to create an xml

In [ ]:
xml = ThinCurr_XML()
xml.set_eta([1, 2, 3e-2])
R_vals = np.linspace(0.1, 1.0, 10)
Z_vals = np.linspace(-0.5, 0.5, 10)
for i, (r, z) in enumerate(zip(R_vals, Z_vals)):
    filament = ThinCurr_Icoil(name=f"Filament_{i}")
    filament.add_subcoil(
        RZ=np.array([[r, z]]),
        scale=1.0
    )
    xml.add_Icoil(filament)

OFT_env.write_oft_xml([xml], "xml_fil_example.xml")
print("Created xml_fil_example.xml")
    

XML file created at xml_fil_example.xml
Done


## Creating basic coil sets
In the following code block we will look at how to create coil sets using the ThinCurr_Icoil and the ThinCurr_Vcoil class that may have multiple coils within each coil set and how to create an xml


In [3]:
xml = ThinCurr_XML()

# Set resistivities
xml.set_eta([5.0e-6, 4e20])

# Create an I-coil with multiple subcoils
icoil1 = ThinCurr_Icoil(name="PF_Coil_1")
icoil1.add_subcoil(
    RZ=np.array([[1.0, 0.5], [1.0, 0.6]]),
    scale=1.0
)
icoil1.add_subcoil(
    RZ=np.array([[1.2, 0.5], [1.2, 0.6]]),
    scale=1.0
)
xml.add_Icoil(icoil1)

# Create another I-coil
icoil2 = ThinCurr_Icoil(name="PF_Coil_2")
icoil2.add_subcoil(
    RZ=np.array([[0.8, -0.5], [0.8, -0.6]]),
    scale=2.0
)
xml.add_Icoil(icoil2)

# Create a V-coil
vcoil1 = ThinCurr_Vcoil(
    name="VCoil_1",
    resistivity_per_length=1.7e-8,
    radius=1.0
)
vcoil1.add_subcoil(
    RZ=np.array([[1.5, 0.0], [1.5, 0.1], [1.5, 0.2]]),
    scale=1.0
)
xml.add_Vcoil(vcoil1)

# Write XML file
OFT_env.write_oft_xml([xml], "xml_coil_example.xml")
print("Created xml_coil_example.xml")


XML file created at xml_coil_example.xml
Created xml_coil_example.xml


## Creating a custom coil xml file

You can also create a custom coil that does not assume axisymmetry, however you must define it as a set of npoints defined by xyz coordinates. You can also set parameters like the restivity per length 

In [4]:
"""Example 2: Custom 3D coils with xyz points"""

# Create ThinCurr XML block
xml = ThinCurr_XML()
xml.set_eta([1.0e-6])

# Create a custom 3D I-coil
icoil_3d = ThinCurr_Icoil(name="Custom_3D_Coil")
xyz_points = np.array([
    [1.0, 0.0, 0.5],
    [0.9, 0.3, 0.5],
    [0.7, 0.7, 0.5],
    [0.3, 0.9, 0.5]
])
icoil_3d.add_subcoil(
    pts=xyz_points,
    scale=1.0,
    npoints=4
)
xml.add_Icoil(icoil_3d)

# Create a custom 3D V-coil
vcoil_3d = ThinCurr_Vcoil(
    name="Custom_Wall",
    resistivity_per_length=1.7e-8,
    radius=1.5,
    sens_mask=1
)
xyz_wall = np.array([
    [1.5, 0.0, 0.0],
    [1.5, 0.0, 0.2],
    [1.5, 0.0, 0.4],
    [1.5, 0.0, 0.6]
])
vcoil_3d.add_subcoil(
    pts=xyz_wall,
    npoints=4
)
xml.add_Vcoil(vcoil_3d)

# Write XML file
OFT_env.write_oft_xml([xml], "xml_3d_coil_example.xml")
print("Created example_3d_coils.xml")



XML file created at xml_3d_coil_example.xml
Created example_3d_coils.xml
